In [2]:
#pip install pyspark

In [3]:
import pyspark
from pyspark import SparkContext

In [18]:
from doctest import master


sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TextClassifierApp").getOrCreate()

In [20]:
df = spark.read.csv("udemy_dataset.csv",header=True,inferSchema=True)

In [36]:
print(df.show(100))

22/09/27 15:35:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , course_id, course_title, url, is_paid, price, num_subscribers, num_reviews, num_lectures, level, content_duration, published_timestamp, subject, clean_course_title
 Schema: _c0, course_id, course_title, url, is_paid, price, num_subscribers, num_reviews, num_lectures, level, content_duration, published_timestamp, subject, clean_course_title
Expected: _c0 but found: 
CSV file: file:///home/wytcor/Stabble-Diffusion-Project/udemy_dataset.csv
+---+---------+--------------------+--------------------+-------+-----+---------------+-----------+------------+------------------+----------------+--------------------+----------------+--------------------+
|_c0|course_id|        course_title|                 url|is_paid|price|num_subscribers|num_reviews|num_lectures|             level|content_duration| published_timestamp|         subject|  clean_course_title|
+---+---------+--------------------+-----------

In [37]:
df.columns

['_c0',
 'course_id',
 'course_title',
 'url',
 'is_paid',
 'price',
 'num_subscribers',
 'num_reviews',
 'num_lectures',
 'level',
 'content_duration',
 'published_timestamp',
 'subject',
 'clean_course_title']

In [51]:
selected_data=df.select('course_title','subject')
selected_data.show()

+--------------------+----------------+
|        course_title|         subject|
+--------------------+----------------+
|Ultimate Investme...|Business Finance|
|Complete GST Cour...|Business Finance|
|Financial Modelin...|Business Finance|
|Beginner to Pro -...|Business Finance|
|How To Maximize Y...|Business Finance|
|Trading Penny Sto...|Business Finance|
|Investing And Tra...|Business Finance|
|Trading Stock Cha...|Business Finance|
|Options Trading 3...|Business Finance|
|The Only Investme...|Business Finance|
|Forex Trading Sec...|Business Finance|
|Trading Options W...|Business Finance|
|Financial Managem...|Business Finance|
|Forex Trading Cou...|Business Finance|
|Python Algo Tradi...|Business Finance|
|Short Selling: Le...|Business Finance|
|Basic Technical A...|Business Finance|
|The Complete Char...|Business Finance|
|7 Deadly Mistakes...|Business Finance|
|Financial Stateme...|Business Finance|
+--------------------+----------------+
only showing top 20 rows



In [50]:
#pip install pandas

In [55]:
##Cheking the dataset integrity

print(selected_data.toPandas()['subject'].isna().sum())

selected_data=selected_data.dropna(subset=('subject'))
print(selected_data.toPandas()['subject'].isna().sum())


0
0


In [57]:
##Feature engineering 
import pyspark.ml.feature

In [58]:
dir(pyspark.ml.feature)

['Any',
 'Binarizer',
 'BucketedRandomProjectionLSH',
 'BucketedRandomProjectionLSHModel',
 'Bucketizer',
 'ChiSqSelector',
 'ChiSqSelectorModel',
 'CountVectorizer',
 'CountVectorizerModel',
 'DCT',
 'DataFrame',
 'DenseMatrix',
 'DenseVector',
 'Dict',
 'ElementwiseProduct',
 'FeatureHasher',
 'Generic',
 'HasFeaturesCol',
 'HasHandleInvalid',
 'HasInputCol',
 'HasInputCols',
 'HasLabelCol',
 'HasMaxIter',
 'HasNumFeatures',
 'HasOutputCol',
 'HasOutputCols',
 'HasRelativeError',
 'HasSeed',
 'HasStepSize',
 'HasThreshold',
 'HasThresholds',
 'HashingTF',
 'IDF',
 'IDFModel',
 'Imputer',
 'ImputerModel',
 'IndexToString',
 'Interaction',
 'JM',
 'JavaEstimator',
 'JavaMLReadable',
 'JavaMLWritable',
 'JavaModel',
 'JavaParams',
 'JavaTransformer',
 'List',
 'MaxAbsScaler',
 'MaxAbsScalerModel',
 'MinHashLSH',
 'MinHashLSHModel',
 'MinMaxScaler',
 'MinMaxScalerModel',
 'NGram',
 'Normalizer',
 'OneHotEncoder',
 'OneHotEncoderModel',
 'Optional',
 'P',
 'PCA',
 'PCAModel',
 'Param',
 '

In [59]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF

In [60]:
from pyspark.ml.feature import StringIndexer

In [ ]:
#Stages to pre-process data

In [61]:
Tokens= Tokenizer(inputCol='course_title', outputCol='words_in_tokens')
stopwords_remover = StopWordsRemover(inputCol='words_in_tokens',outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [83]:
labelEncoder = StringIndexer(inputCol='subject',outputCol='label').fit(selected_data)

#print(labelEncoder.labels)

labelEncoder.transform(selected_data).show(15)

+--------------------+----------------+-----+
|        course_title|         subject|label|
+--------------------+----------------+-----+
|Ultimate Investme...|Business Finance|  1.0|
|Complete GST Cour...|Business Finance|  1.0|
|Financial Modelin...|Business Finance|  1.0|
|Beginner to Pro -...|Business Finance|  1.0|
|How To Maximize Y...|Business Finance|  1.0|
|Trading Penny Sto...|Business Finance|  1.0|
|Investing And Tra...|Business Finance|  1.0|
|Trading Stock Cha...|Business Finance|  1.0|
|Options Trading 3...|Business Finance|  1.0|
|The Only Investme...|Business Finance|  1.0|
|Forex Trading Sec...|Business Finance|  1.0|
|Trading Options W...|Business Finance|  1.0|
|Financial Managem...|Business Finance|  1.0|
|Forex Trading Cou...|Business Finance|  1.0|
|Python Algo Tradi...|Business Finance|  1.0|
+--------------------+----------------+-----+
only showing top 15 rows



In [73]:
label_dict = {'Web Development':0.0,
 'Business Finance':1.0,
 'Musical Instruments':2.0,
 'Graphic Design':3.0}

In [84]:
pos_processed=labelEncoder.transform(selected_data)
pos_processed.show()

+--------------------+----------------+-----+
|        course_title|         subject|label|
+--------------------+----------------+-----+
|Ultimate Investme...|Business Finance|  1.0|
|Complete GST Cour...|Business Finance|  1.0|
|Financial Modelin...|Business Finance|  1.0|
|Beginner to Pro -...|Business Finance|  1.0|
|How To Maximize Y...|Business Finance|  1.0|
|Trading Penny Sto...|Business Finance|  1.0|
|Investing And Tra...|Business Finance|  1.0|
|Trading Stock Cha...|Business Finance|  1.0|
|Options Trading 3...|Business Finance|  1.0|
|The Only Investme...|Business Finance|  1.0|
|Forex Trading Sec...|Business Finance|  1.0|
|Trading Options W...|Business Finance|  1.0|
|Financial Managem...|Business Finance|  1.0|
|Forex Trading Cou...|Business Finance|  1.0|
|Python Algo Tradi...|Business Finance|  1.0|
|Short Selling: Le...|Business Finance|  1.0|
|Basic Technical A...|Business Finance|  1.0|
|The Complete Char...|Business Finance|  1.0|
|7 Deadly Mistakes...|Business Fin

In [100]:
##Split dataset

(trainDF,testDF) = pos_processed.randomSplit((0.7,0.3),seed=1)

In [101]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [102]:
##Pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[Tokens,stopwords_remover,vectorizer,idf,lr])

In [103]:
lr_model = pipeline.fit(trainDF)

In [104]:
predictions = lr_model.transform(testDF)

In [105]:
predictions.columns

['course_title',
 'subject',
 'label',
 'words_in_tokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [106]:
predictions.select('course_title','subject','label','prediction').show()

+--------------------+-------------------+-----+----------+
|        course_title|            subject|label|prediction|
+--------------------+-------------------+-----+----------+
|#5  Piano Hand Co...|Musical Instruments|  2.0|       2.0|
|#6 Piano Hand Coo...|Musical Instruments|  2.0|       2.0|
|         1 Hour HTML|    Web Development|  0.0|       0.0|
|   1 Hour JavaScript|    Web Development|  0.0|       0.0|
|       1 hour jQuery|    Web Development|  0.0|       0.0|
|1. Principles of ...|   Business Finance|  1.0|       1.0|
|10 Numbers Every ...|   Business Finance|  1.0|       1.0|
|3 Little Pigs: A ...|   Business Finance|  1.0|       1.0|
|3. Compound Inter...|   Business Finance|  1.0|       1.0|
|30 Day Guitar Jum...|Musical Instruments|  2.0|       2.0|
|3D Programming wi...|    Web Development|  0.0|       0.0|
|4. Ordinary Simpl...|   Business Finance|  1.0|       1.0|
|5 lecciones que t...|Musical Instruments|  2.0|       1.0|
|6 Must Know Trick...|Musical Instrument

#Evaluation

In [107]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label', metricName='accuracy')

print(evaluator.evaluate(predictions))

0.9238440616500453
